# Задание 4

## 1. Декоратор @cached (0.3 балла)

#### Реализуйте класс для хранения результатов выполнения функции

* max_count - максимальное число хранимых результатов. Если число результатов превышает max_count, требуется выбросить первый результат, т. е. в кеше должно храниться не более max_count последних результатов.
* продумайте архитектуру кеша так, чтобы для функций:

<code>
@cached
def f1():
    pass

@cached
def f2():
    pass
</code>    
должны иметь по max_count хранимых последних результатов, и т. д.

<b>P. S.</b>

* Считайте, что функция не имеет состояния (зависит только от передаваемых в нее аргументов).
* Храните данные так, чтобы из функции нельзя напрямую было получить закешированные результаты (только через \_\_closer\_\_).

<b>Рекомендации:</b>

* Для хранения данных используйте OrderedDict.
* Декорируйте wrapper с @functools.wraps(func)

In [1]:
import functools
from collections import OrderedDict

In [2]:
class imdict(dict):
    def __hash__(self):
        return id(self)

    def _immutable(self, *args, **kws):
        raise TypeError('object is immutable')

In [3]:
import collections

class LruCache(object):
    def __init__(self, max_count):
        self.max_count = max_count
        self.cache = OrderedDict()

    def __getitem__(self, key):
        return self.cache.get(key)

    def __setitem__(self, key, value):
        if len(self.cache) == self.max_count:
            self.cache.popitem(0)
        self.cache[key] = value

#### Реализуйте декоратор

In [4]:
def cached(max_count):
    def cached_decorator(func):
        cache = LruCache(max_count)
        
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key = str(imdict({"args": args, "kwargs": kwargs}))
            
            cached_value = cache[key]
            if cached_value == None:
                cached_value = func(*args, **kwargs)
                cache[key] = cached_value

            return cached_value
        return wrapper
    return cached_decorator

#### Проверьте использование декоратора

In [5]:
@cached(32)
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

@cached(32)
def fact1(n):
    if n < 2:
        return 1
    return fact1(n-1) * n

In [6]:
print(fact(20))
print(fact(20))

2432902008176640000
2432902008176640000


#### Сравните свою реализацию с lru_cache из functools

In [7]:
from functools import lru_cache

In [8]:
@cached(32)
def fact_non_rec(n):
    ans = 1
    for i in range(1, n + 1):
        ans *= i
    return ans

@lru_cache(maxsize=32)
def lru_fact_non_rec(n):
    ans = 1
    for i in range(1, n + 1):
        ans *= i
    return ans

In [9]:
value = 20000
times = 1000000

** Проверим работу `lru_cache` **

In [10]:
%time a = lru_fact_non_rec(value)

CPU times: user 261 ms, sys: 7.17 ms, total: 268 ms
Wall time: 284 ms


In [11]:
%time for i in range(times): lru_fact_non_rec(value)

CPU times: user 557 ms, sys: 9.27 ms, total: 566 ms
Wall time: 588 ms


** Теперь посмотрим на свою реализацию **

In [12]:
%time a = fact_non_rec(value)

CPU times: user 265 ms, sys: 5.06 ms, total: 270 ms
Wall time: 290 ms


In [13]:
%time for i in range(times): fact_non_rec(value)

CPU times: user 5.91 s, sys: 47.3 ms, total: 5.95 s
Wall time: 6.13 s


** Из значений видно, что предложенная реализация работает заметно медленней, однако значительно быстрее чем вычисление функции большое количество раз **

### Дополнительное задание (0.2 балла)

Дополните декоратор @cached так, чтобы не пересчитывать функцию при изменения ее состояния (например, она использовала глобальную переменную)

In [15]:
def cached_globals(max_count):
    def cached_decorator(func):
        cache = LruCache(max_count)
        
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key = str(imdict({"args": args, "kwargs": kwargs, "globals": globals()}))
            
            cached_value = cache[key]
            if cached_value == None:
                cached_value = func(*args, **kwargs)
                cache[key] = cached_value
            else:
                print("Restored value:", end=" ")
            return cached_value
        return wrapper
    
    return cached_decorator

In [16]:
a = 0

@cached_globals(20)
def getA():
    return a

In [17]:
print(getA())
a = 10
print(getA())
print(getA())

0
10
Restored value: 10


## 2. Декоратор @checked (0.3 балла)

Напишите декоратор, который будет вызывать исключение (raise TypeError), если в него переданы аргументы не тех типов.

<b>P. S.</b> Разберитесь с модулем typing.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Чтобы кинуть иключение используйте конструкцию типа:
<code>
if < some_condtion >:
    raise TypeError
</code>

In [18]:
def checked(*types):
    def checked_decorator(func):        
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            for i, (arg, exp_type) in enumerate(zip(args, types)):
                if type(arg) != exp_type:
                    raise TypeError("parameter {} must be a '{}'".format(i, exp_type.__name__))
            return func(*args, **kwargs)
        return wrapper
    return checked_decorator

#### Проверьте использование декоратора

In [19]:
from typing import List

# Пример
@checked(str, int, list)
def strange_func(a: str, b: int, c: List):
    pass

In [20]:
strange_func("test", 1, [3, 3, 4])

In [21]:
strange_func("test", 1, 3)

TypeError: parameter 2 must be a 'list'

In [22]:
strange_func("sd", 1)

TypeError: strange_func() missing 1 required positional argument: 'c'

## 3. Декоратор @Logger (0.4 балла)

Напишите полноценный logger для вызовов вашей функции. Декоратор должен иметь следующие опции:

* Выбор файла в который будет производиться запись: sys.stdout, sys.stderr, локальный файл (передается путь к файлу, если файла нет, то создать, иначе дописывать в конец).
* Формат записи в логера: "<i>index data time functio_name \*args \**kwargs result</i>"
* Логер должен быть один для всех функций.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Создайте отдельный класс Logger для работы с выводом данных вызовов функций в файл.

In [23]:
import datetime
class Logger:
    index = 0
        
    def log(stream, function_name, args, kwargs, res):
        is_file = False
        if isinstance(stream, str):
            is_file = True
            stream = open(stream, "a")

        now = datetime.datetime.now()
        stream.write("{} {} {} {} {} {}\n".format(Logger.index, 
                                                  now.strftime("%Y-%m-%d %H:%M"),
                                                  function_name, 
                                                  args, 
                                                  kwargs,
                                                  res)
                        )
            
        if is_file:
            stream.close()

        Logger.index += 1

In [24]:
def logger(stream):
    def logger_decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):

            res = func(*args, **kwargs)

            Logger.log(stream, func.__name__, args, kwargs, res)
            return res

        return wrapper
    return logger_decorator

In [25]:
@logger("logger.txt")
def to_file(a, b):
    return a + b

import sys
@logger(sys.stdout)
def to_std(a, b):
    return a + b

In [26]:
for i in range(10):
    to_file(1, 1)
    to_std(1, 1)

1 2018-03-29 23:55 to_std (1, 1) {} 2
3 2018-03-29 23:55 to_std (1, 1) {} 2
5 2018-03-29 23:55 to_std (1, 1) {} 2
7 2018-03-29 23:55 to_std (1, 1) {} 2
9 2018-03-29 23:55 to_std (1, 1) {} 2
11 2018-03-29 23:55 to_std (1, 1) {} 2
13 2018-03-29 23:55 to_std (1, 1) {} 2
15 2018-03-29 23:55 to_std (1, 1) {} 2
17 2018-03-29 23:55 to_std (1, 1) {} 2
19 2018-03-29 23:55 to_std (1, 1) {} 2
